In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [377]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.1, 0.9]
)

In [472]:
IMAGE_SIZE = 150
batch_size = 10
train_generator = train_datagen.flow_from_directory(
    'C:/Users/Rana Wael/Documents/Slash/data/validation',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True, 
    seed=42
)

Found 31 images belonging to 5 classes.


In [457]:
validation_generator = train_datagen.flow_from_directory(
    'C:/Users/Rana Wael/Documents/Slash/data/train',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 196 images belonging to 5 classes.


In [458]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory('C:/Users/Rana Wael/Documents/Slash/data/test', 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 6 images belonging to 5 classes.


In [473]:
from tensorflow.keras import regularizers
dropout_conv=0.3
dropout_dense=0.5
model = Sequential()
model.add(Conv2D(32, (3, 3),input_shape = (IMAGE_SIZE,IMAGE_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(dropout_dense))
model.add(Dense(5, activation='softmax'))

model.compile( loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
fitting = model.fit(train_generator, steps_per_epoch=200//batch_size, validation_data=validation_generator, validation_steps=800//batch_size, epochs=30, verbose=1)

Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.2307 - loss: 3.9184 - val_accuracy: 0.1837 - val_loss: 1.6561
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.2720 - loss: 1.4290 - val_accuracy: 0.1837 - val_loss: 1.5317
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2253 - loss: 1.3809 - val_accuracy: 0.2041 - val_loss: 1.5288
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2001 - loss: 1.3520 - val_accuracy: 0.2194 - val_loss: 1.5265
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.3314 - loss: 1.3311 - val_accuracy: 0.2041 - val_loss: 1.5263
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.1765 - loss: 1.3387 - val_accuracy: 0.2041 - val_loss: 1.5222
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2993 - loss: 1.3471 - val_accuracy: 0.1480 - val_loss: 1.5440
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2003 - loss: 1.3617 - val_accuracy: 0.3673 - val_loss:

In [ ]:
fig = plt.figure()
plt.plot(fitting.history['accuracy'], color ='blue', label = 'accuracy')
plt.plot(fitting.history['val_accuracy'], color ='orange', label = 'val_accuracy')
fig.title('Accuracy', fontsize=20)
plt.show()


In [ ]:
from tensorflow.keras.applications import VGG16

# Load pre-trained VGG16 model (excluding the top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(CLASSES, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(train_generator, epochs=20, batch_size=16, validation_data=validation_generator)

# Unfreeze the top layers of the base model for fine-tuning
base_model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), metrics=['accuracy'])

model.fit(train_generator, validation_data=validation_generator, epochs=20, batch_size=32)

In [ ]:
fig = plt.figure()
plt.plot(fitting.history['accuracy'], color ='blue', label = 'accuracy')
plt.plot(fitting.history['val_accuracy'], color ='orange', label = 'val_accuracy')
fig.title('Accuracy', fontsize=20)
plt.show()


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)